# Google Landmarks Startup Notebook

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np

import urllib
import cv2

In [ ]:
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader

# Prepare Data

In [ ]:
base_path = '../input/google-landmarks-dataset/'

# load train data
df_train = pd.read_csv(base_path + 'train.csv')
print(df_train.head())

# load boxes data and merge into one
df_boxes_split1 = pd.read_csv(base_path + 'boxes_split1.csv')
df_boxes_split2 = pd.read_csv(base_path + 'boxes_split2.csv')
df_boxes = pd.concat([df_boxes_split1, df_boxes_split2])

print(df_boxes.head())

In [ ]:
# merge train and boxes on id
df_train = pd.merge(df_train, df_boxes, on='id',  how='right')
df_train.head()

# Augmentations

In [ ]:
def get_transform(train):
    transforms = []
    if train:
        # random horizontal flip with 50% probability
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# Google Landmarks Dataset

In [ ]:
class GoogleLandmarks(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.dim = (512, 512)
        self.transforms = transforms
        self.ids = np.unique(df['id'].values)
        self.ids_dic = {k:v for k,v in enumerate(self.ids)}
    
    def url_to_image(self, url, dim):
        try:
            resp = urllib.request.urlopen(url)
        except:
            return np.array([])
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        if(image.size != 0):
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            image = Image.fromarray(np.uint8(image))
            if(image):
                image = self.transforms(image)
        return image
    
    def get_rect(self, boxes):
        try:
            y = boxes[0]
            x = boxes[1]
            h = boxes[2] - boxes[0]
            w = boxes[3] - boxes[1]
        except:
            return None
        return plt.Rectangle((x, y), w, h, color='y', alpha=0.3)
    
    def draw_bbox(self, img, rect):
        fig, ax = plt.subplots() 
        plt.imshow(img)
        if(rect):
            ax.add_patch(rect)
    
    def format_boxes(self, boxes, dim):
        return (np.array(boxes.split(' ')).astype(np.float32) * dim[0]).astype(np.int64)
    
    def __getitem__(self, idx):
        id = self.ids_dic[idx]
        url = self.df[self.df.id == id].url.values[0]
        boxes = self.df[self.df.id == id].box.values[0]
        
        
        # format boxes
        boxes = self.format_boxes(boxes, self.dim)
        
        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.int64)
        target["labels"] = torch.ones((1,), dtype=torch.int64)
        target["image_id"] = torch.tensor([idx])
        target["area"] = (boxes[3] - boxes[1]) * (boxes[2] - boxes[0])
        target["iscrowd"] = torch.zeros((1,), dtype=torch.int64)
        
        image = self.url_to_image(url, self.dim)
        
        if(image is None):
            return None, None
 
        return image, target
        
    def __len__(self):
        return len(self.ids)

In [ ]:
google_ds = GoogleLandmarks(df_train, get_transform(train=True))

# Draw an image with Bounding Boxes

In [ ]:
image, target = google_ds[2]

In [ ]:
rect = google_ds.get_rect(target['boxes'])
google_ds.draw_bbox(image, rect)